# TFM Modelo predicción de índice de gravedad

In [ ]:
# Carga de datos
import pandas as pd
data = pd.read_excel("df_final_con_gravedad.xlsx")


In [ ]:
data.head()

## **Procesamiento de datos**

En esta etapa se llevará a cabo la preparación y depuración del conjunto de datos, con el objetivo de dejarlo en condiciones óptimas para la fase de entrenamiento del modelo. Este paso es fundamental, ya que influye de manera directa en el rendimiento y la calidad de los resultados obtenidos.

In [ ]:
# Se asegura que dia_hora está en formato datetime
data['dia_hora'] = pd.to_datetime(data['dia_hora'])

# Separar en dos columnas para luego hacer la clasificacion de franja horaria
data['fecha'] = data['dia_hora'].dt.date
data['hora'] = data['dia_hora'].dt.time



In [ ]:
# Se eliminan las columnas que no aporten informacion al modelo
drop_columns = [
    'num_expediente',
    'dia_hora',
    'tipo_accidente',
    "Peatones",
    "total_implicados",
    "diversidad_vehiculos",
    "categoria_gravedad",
    "Conductores"
]
data.drop(drop_columns, axis=1, inplace=True)

In [ ]:
#La columna de estado meteorológico se separa en nuevas columnas de conteo de cada precipitación
data["Despejado"]=(data["estado_meteorológico"]=="Despejado").astype(int)
data["Lluvia débil"]=(data["estado_meteorológico"]=="Lluvia débil").astype(int)
data["Nublado"]=(data["estado_meteorológico"]=="Nublado").astype(int)
data["Lluvia intensa"]=(data["estado_meteorológico"]=="Lluvia intensa").astype(int)
data['Granizando']=(data["estado_meteorológico"]=='Granizando').astype(int)
data['Nevando']=(data["estado_meteorológico"]=='Nevando').astype(int)

data=data.drop(columns=["estado_meteorológico"])

In [ ]:
# Agrupar por 'distrito', 'fecha' y 'franja_horaria' y aplicamos funciones de agregación.
agrupacion_completa = data.groupby(['distrito', 'fecha', 'franja_horaria']).agg(
    total_pasajeros=('Pasajeros', 'median'),
    vehiculo_dos_ruedas=('Vehículo de dos ruedas', lambda x: int((x != 0).any())),
    vehiculo_pesado=('Vehículo pesado', lambda x: int((x != 0).any())),
    turismo=('Turismo', lambda x: int((x != 0).any())),
    otros_vehiculos=('Otros vehículos', lambda x: int((x != 0).any())),
    tiene_vulnerables=('tiene_vulnerables', 'sum'),
    indice_gravedad=('indice_gravedad', 'mean'),
    Despejado=('Despejado', 'sum'),
    Lluvia_débil=('Lluvia débil', 'sum'),
    Lluvia_intensa=('Lluvia intensa', 'sum'),
    Nublado=('Nublado', 'sum'),
    Granizo= ('Granizando', 'sum'),
    Nevando= ('Nevando', 'sum'),
).reset_index()

In [ ]:
agrupacion_completa.head(10)

In [ ]:
#Se compara el numero de filas esperado (nº distritos por las 4 franjas horarias y por todos los dias de año)
# Se renombra como df para simplificar
df = agrupacion_completa

# Valores únicos
distritos = df['distrito'].unique()
fechas = pd.date_range(start='2021-01-01', end='2024-12-31', freq='D')
franjas = ['Madrugada', 'Mañana', 'Tarde', 'Noche']

# Total esperado
total_esperado = len(distritos) * len(fechas) * len(franjas)
print(f"Registros esperados: {total_esperado}")
print(f"Registros actuales: {len(df)}")


Al agrupar los datos por fecha (del 1 de enero de 2021 al 31 de diciembre de 2024), distrito (21 distritos de Madrid) y franja horaria (4 intervalos definidos), el número esperado de registros en el dataset es de 122,724. Sin embargo, el resultado obtenido es de 54,179 registros.

Esta diferencia se explica porque no todos los días, en cada distrito y en cada franja horaria, ocurren accidentes. En consecuencia, al realizar la agregación, aquellas combinaciones en las que no se registra ningún accidente no aparecen reflejadas en el dataset.

La solución consiste en generar el producto cartesiano de todas las combinaciones posibles entre fecha, distrito y franja horaria, para posteriormente realizar un merge con el dataset original. De esta forma, las combinaciones ausentes se incorporan y las columnas correspondientes a los accidentes se completan con valores 0, garantizando así una representación consistente y completa de todas las posibles combinaciones.

In [ ]:
import itertools
df['fecha'] = pd.to_datetime(df['fecha'])

# Producto cartesiano de todas las combinaciones posibles
combinaciones = pd.DataFrame(itertools.product(distritos, fechas, franjas),
                             columns=['distrito', 'fecha', 'franja_horaria'])
df_completo = pd.merge(combinaciones, df,
                       on=['distrito', 'fecha', 'franja_horaria'],
                       how='left')
columnas_a_rellenar = [

    'total_pasajeros',
    'vehiculo_dos_ruedas',
    'vehiculo_pesado',
    'turismo', 'otros_vehiculos',
    "indice_gravedad",
    'Despejado', 'Lluvia_débil', 'Lluvia_intensa',
    'Nublado','Granizo', 'Nevando'
]

# Se rellenan NaNs con 0 en estas columnas
df_completo[columnas_a_rellenar] = df_completo[columnas_a_rellenar].fillna(0)
columnas_a_ajustar = [

    'total_pasajeros',
    'vehiculo_dos_ruedas',
    'vehiculo_pesado',
    "turismo",
    'otros_vehiculos',
    "tiene_vulnerables"
]

df_completo.loc[df_completo['indice_gravedad'] == 0, columnas_a_ajustar] = 0

df_completo.head()
print(f"Registros actuales: {len(df_completo)}")

In [ ]:
# Creamos la variable día_semana
dias_esp = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
df_completo['dia_semana'] = df_completo['fecha'].dt.day_name().map(dias_esp)

In [ ]:
# Se separa la fecha en mes y dia, esto ya que el modelo no aprende de formato datetime
# Eliminamos el año ya que el modelo va a predecir con un año distinto al de aprendizaje
# Aseguramos de nuevo que la variable fecha está en formato correcto
df_completo["fecha"] = pd.to_datetime(df_completo["fecha"])

# Extrae año, mes y día
df_completo["mes"] = df_completo["fecha"].dt.month
df_completo["día"] = df_completo["fecha"].dt.day




In [ ]:
#Ahora vamos a crear una columna llamada festivo, que nos dice si es un día festivo o no
!pip install holidays
import holidays

festivos = holidays.country_holidays('ES', subdiv='MD')
df_completo['es_festivo'] = df_completo['fecha'].dt.date.apply(lambda x: 1 if x in festivos else 0)

#ELiminamos fecha
df_completo=df_completo.drop(columns=["fecha"])

In [ ]:
df_completo.info()

## Creación del modelo

In [ ]:

from sklearn.model_selection import train_test_split

# Se detectan las columnas categóricas automáticamente (dtype == object)
categorical_cols = df_completo.select_dtypes(include=['object']).columns.tolist()

# Se elimina la variable indice_gravedad de la codificación
if 'indice_gravedad' in categorical_cols:
    categorical_cols.remove('indice_gravedad')

# Se aplica One-Hot Encoding con pandas
data_encoded = pd.get_dummies(df_completo, columns=categorical_cols, drop_first=True)

In [ ]:
# Se separan los datos en X_data e y_data
X_data = data_encoded.drop("indice_gravedad", axis=1)
y_data = data_encoded["indice_gravedad"]

In [ ]:
# Train test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, random_state=42, shuffle= True)


In [ ]:
# IMportaciones para el modeladp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Instalamos xgboost si fuera necesario e importamos
!pip install xgboost
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Convertir datos a DMatrix para la entrada de los datos al modelo xgboost
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

params = {
    'objective': 'reg:squarederror',
    'max_depth': 6,
    'subsample': 0.8,
    'seed': 42
}

evals = [(dval, 'eval'), (dtrain, 'train')]

model_xb = xgb.train(params, dtrain, num_boost_round=100,
                  evals=evals,
                  early_stopping_rounds=10,
                  verbose_eval=True)



In [ ]:
# Predicción
y_train_pred = model_xb.predict(dtrain)
y_val_pred = model_xb.predict(dval)

# Métricas
mse_train = mean_squared_error(y_train, y_train_pred)
mae_train = mean_absolute_error(y_train, y_train_pred)

mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_val = r2_score(y_val, y_val_pred)

print(f"Train MSE: {mse_train:.4f}, MAE: {mae_train:.4f}")
print(f"Val MSE: {mse_val:.4f}, MAE: {mae_val:.4f}")
print(f"Train R2: {r2_train:.4f}, Val R2: {r2_val:.4f}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Obtenemos importancias desde el Booster
scores = model_xb.get_score(importance_type='gain')  # también puedes usar 'weight' o 'cover'

# Convertimos a DataFrame
importancia_df = pd.DataFrame({
    'Variable': list(scores.keys()),
    'Importancia': list(scores.values())
}).sort_values(by='Importancia', ascending=False)

# Mostramos
print(importancia_df)

# Gráfico
plt.figure(figsize=(10,6))
plt.barh(importancia_df['Variable'], importancia_df['Importancia'], color='skyblue')
plt.xlabel('Importancia')
plt.title('Importancia de las variables en XGBoost (gain)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()



## Sistema de widgets para simulacro de funcionamiento

In [ ]:
# Se importan los datos
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import unicodedata  # para normalizar y quitar tildes

# Se cran listan con los los valores de los widgets

distritos = [
    'CENTRO', 'ARGANZUELA', 'RETIRO', 'SALAMANCA', 'CHAMARTÍN', 'TETUÁN',
    'CHAMBERÍ', 'FUENCARRAL-EL PARDO', 'MONCLOA-ARAVACA', 'LATINA', 'CARABANCHEL',
    'USERA', 'PUENTE DE VALLECAS', 'MORATALAZ', 'CIUDAD LINEAL', 'HORTALEZA',
    'VILLAVERDE', 'VILLA DE VALLECAS', 'VICÁLVARO', 'SAN BLAS-CANILLEJAS', 'BARAJAS'
]

franjas_horarias = ['Madrugada', 'Mañana', 'Tarde', 'Noche']

climas = ['Despejado', 'Lluvia_débil', 'Lluvia_intensa', 'Nublado', 'Granizo', 'Nevando']

dias_semana = [
    'Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo'
]

# Se definen los widgets

franja_horaria = widgets.Dropdown(options=franjas_horarias, description='Franja:')
pasajeros = widgets.FloatSlider(min=0, max=50, step=1, description='Pasajeros:')
vehiculo_dos_ruedas = widgets.FloatSlider(min=0, max=1, step=1, description='2 ruedas:')
vehiculo_pesado = widgets.FloatSlider(min=0, max=1, step=1, description='Pesados:')
turismo = widgets.FloatSlider(min=0, max=1, step=1, description='Turismos:')
otros_vehiculos = widgets.FloatSlider(min=0, max=1, step=1, description='Otros veh.:')
mes = widgets.IntSlider(min=1, max=12, description='Mes:')
dia = widgets.IntSlider(min=1, max=31, description='Día:')
es_festivo= widgets.Checkbox(value=False, description="Festivo")
dia_semana = widgets.Dropdown(options=dias_semana, description='Día de la semana:')
tiene_vulnerables = widgets.Checkbox(value=False, description='¿Vulnerables?')



clima_widgets = {clima: widgets.Checkbox(value=False, description=clima) for clima in climas}
boton = widgets.Button(description="Predecir accidentes por distrito")

inputs = widgets.VBox([
    franja_horaria,
    pasajeros,
    vehiculo_dos_ruedas, vehiculo_pesado, turismo, otros_vehiculos,
    mes, dia, dia_semana, es_festivo, tiene_vulnerables,
    widgets.HTML(value="<b>Condiciones climáticas:</b>"),
    *clima_widgets.values(), boton
])

display(inputs)

# Se cargan los datos geográficos

url = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/madrid-districts.geojson"
gdf = gpd.read_file(url)
gdf.rename(columns={"name": "NOMBRE"}, inplace=True)

def normalizar_nombre(s):
    s = s.upper()
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')  # quita tildes
    s = s.replace('-', ' ')
    s = s.strip()
    if s == "SAN BLAS CANILLEJAS":
        s = "SAN BLAS"
    return s

gdf["NOMBRE_norm"] = gdf["NOMBRE"].apply(normalizar_nombre)

# Función para mostrar el mapa

def mostrar_mapa(df_pred):
    df_pred['distrito_norm'] = df_pred['distrito'].apply(normalizar_nombre)
    df_merge = gdf.merge(df_pred, left_on='NOMBRE_norm', right_on='distrito_norm', how='left')

    min_pred = df_merge["predicción_Índice_Gravedad"].min()
    df_merge['escala_color'] = df_merge["predicción_Índice_Gravedad"] / (min_pred if min_pred != 0 else 1)

    cmap = LinearSegmentedColormap.from_list(
        "azul_madrid",
        ["#dbe9f6", "#6baed6", "#08519c"]
    )

    fig, ax = plt.subplots(figsize=(10, 10))
    df_merge.plot(column='escala_color', cmap=cmap, linewidth=0.8, edgecolor='0.8', legend=False, ax=ax)

    # Añadir los valores predichos en el centro de cada polígono
    for idx, row in df_merge.iterrows():
        if pd.notnull(row["predicción_Índice_Gravedad"]):
            centroid = row["geometry"].centroid
            ax.text(centroid.x, centroid.y,
                    f'{row["predicción_Índice_Gravedad"]:.2f}',
                    horizontalalignment='center',
                    fontsize=8,
                    color='black')

    # Calcular la media y generar el mensaje
    media = df_merge["predicción_Índice_Gravedad"].mean()
    if 2 <= media < 3.5:
        mensaje = ("El nivel de riesgo se considera moderado;\n"
        "se recomienda mantener medidas básicas de precaución")
    elif 3.5 <= media < 5.5:
        mensaje = ("Se detecta un nivel de riesgo relevante;\n"
        "se aconseja aumentar la atención durante la conducción.")
    elif media >= 5.5:
        mensaje = ("El nivel de riesgo es elevado;\n"
        "se recomienda extremar las precauciones y evitar zonas de alta concentración vehicular")
    else:
        mensaje = ("El nivel de riesgo es bajo;\n"
        "no se requieren medidas adicionales.")

    # Añadir el texto debajo del mapa
    plt.figtext(0.5, 0.02,
                f"El índice de gravedad promedio es de {media:.2f}. {mensaje}.",
                ha='center', fontsize=14)

    ax.set_title("Predicción de accidentes por distrito", fontsize=16)
    ax.axis("off")
    plt.show()

feature_names = X_train.columns.tolist()  # variable global o en scope accesible

def on_button_clicked(b):
    rows = []
    for distrito in distritos:
        row = {
            'distrito': distrito,
            'total_pasajeros': pasajeros.value,
            'mes': mes.value,
            'dia': dia.value,
            'es_festivo': es_festivo.value,
            'dia_semana': dia_semana.value,
            'tiene_vulnerables': tiene_vulnerables.value,
            'franja_horaria': franja_horaria.value,
            'vehiculo_dos_ruedas': vehiculo_dos_ruedas.value,
            'vehiculo_pesado': vehiculo_pesado.value,
            'turismo': turismo.value,
            'otros_vehiculos': otros_vehiculos.value,
        }
        for clima in clima_widgets:
            row[clima] = int(clima_widgets[clima].value)
        rows.append(row)

    df_input = pd.DataFrame(rows)

    # One-hot encoding
    df_encoded = pd.get_dummies(df_input)

    # Añadir columnas faltantes con ceros
    for col in feature_names:
        if col not in df_encoded.columns:
            df_encoded[col] = 0

    # Reordenar columnas para coincidir con el entrenamiento
    df_encoded = df_encoded[feature_names]

    # Crear DMatrix para la predicción
    dmatrix_input = xgb.DMatrix(df_encoded, feature_names=feature_names)

    # Predecir con Booster
    df_input["predicción_Índice_Gravedad"] = model_xb.predict(dmatrix_input)

    # Mostrar resultados
    display(df_input[['distrito', "predicción_Índice_Gravedad"]].sort_values(by="predicción_Índice_Gravedad", ascending=False))
    mostrar_mapa(df_input[['distrito', "predicción_Índice_Gravedad"]])


# --- Conectar botón ---
boton.on_click(on_button_clicked)
